In [1]:
import pandas as pd
import pyam
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import aneris

In [2]:
mimosa_path = '../../../data/TOD/model_results/raw/MIMOSA_processed_2.xlsx'

In [3]:
def round_to_0_1_percent(value):
    if value == 0:
        return 0
    # Determine the number of significant digits to round to
    magnitude = np.floor(np.log10(abs(value))) - 2
    rounding_factor = 10 ** magnitude
    return np.round(value / rounding_factor) * rounding_factor

In [4]:
df_unclean = pd.read_excel(mimosa_path)
ratio_mask = df_unclean['Unit'] == 'fraction of GDP'
# Apply rounding to the 'value' column for rows where the 'unit' column is '%'
df_unclean.loc[ratio_mask, df_unclean.columns[5:]] = df_unclean.loc[ratio_mask, df_unclean.columns[5:]]  * 100
df_unclean.loc[ratio_mask, 'Unit'] = "% of GDP"

damage_costs_mask = (df_unclean['Variable'] == 'Damage costs') & (df_unclean['Unit'] == "% of GDP")
df_unclean.loc[damage_costs_mask, 'Variable'] = "Damage Costs|Relative"

df_unclean.Variable = df_unclean.Variable.replace({'Damage costs':'Damage Costs',
                                                   'Damage costs|Indirect':'Damage Costs|Indirect'})

# df_unclean.loc[:, df_unclean.columns[5:]] = df_unclean.loc[:, df_unclean.columns[5:]].applymap(round_to_0_1_percent)

df_unclean

,Model,Scenario,Region,Variable,Unit,2020,2030,2040,2050,2060,2070,2080,2090,2100
0,MIMOSA,REF,World,GDP|Climate Damage Corrected,trillion USD_2010/yr,102.473673,149.737147,193.516256,237.306060,282.372230,330.966820,379.361690,426.157800,469.749200
1,MIMOSA,REF,World,GDP|Climate Damage Corrected|Loss,trillion USD_2010/yr,0.000000,1.272265,4.171992,9.488870,18.562610,33.532070,56.395150,89.055100,133.434820
2,MIMOSA,REF,World,GDP|Climate Damage Corrected|Loss|Relative,% of GDP,0.000000,0.842507,2.110389,3.844840,6.168315,9.199499,12.941885,17.285107,22.121743
3,MIMOSA,REF,World,Damage Costs|Relative,% of GDP,0.000000,0.682880,1.685171,3.038670,4.842273,7.196702,10.109326,13.501829,17.310141
4,MIMOSA,REF,World,Damage Costs,trillion USD_2010/yr,0.000000,1.029555,3.316975,7.436939,14.369029,25.665775,42.663958,66.520603,98.336391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,MIMOSA,LIFE-TP,RSAF,GDP|Climate Damage Corrected|Loss,trillion USD_2010/yr,0.000000,0.011051,0.031980,0.051290,0.069770,0.088800,0.107610,0.123000,0.127600
434,MIMOSA,LIFE-TP,RSAF,GDP|Climate Damage Corrected|Loss|Relative,% of GDP,0.000000,1.396007,2.509692,2.399252,1.961876,1.561601,1.245186,0.983685,0.738854
435,MIMOSA,LIFE-TP,RSAF,Avoided Damages,trillion USD_2010/yr,0.000000,0.002521,0.021680,0.106800,0.332100,0.821880,1.741930,3.284200,5.648600
436,MIMOSA,LIFE-TP,RSAF,Avoided Damages|Relative,% of GDP,0.000000,0.318463,1.701380,4.995907,9.338384,14.453255,20.156374,26.265195,32.707585


In [5]:
def compute_UN_groups(df, variables, weight=None):
    
    df_africa = df.aggregate_region(variables,
                              region='Africa (UN-R5)',
                              subregions=[
                                    'NAF',
                                    'WAF',
                                    'EAF',
                                    'SAF',
                                    'RSAF',
                              ],
                              weight=weight
                              )

    df_asia = df.aggregate_region(variables,
                              region='Asia and the Pacific (UN-R5)',
                              subregions=[
                                'CHN',
                                'INDIA',
                                'INDO',
                                'JAP',
                                'KOR',
                                'ME',
                                'RSAS',
                                'SEAS',
                                'STAN',
                              ],
                              weight=weight
                              )

    df_ee = df.aggregate_region(variables,
                              region='Eastern Europe (UN-R5)',
                              subregions=[
                                    'CEU',
                                    'RUS',
                                    'UKR',
                              ],
                              weight=weight
                              )

    df_latin = df.aggregate_region(variables,
                              region='Latin America and Caribbean (UN-R5)',
                              subregions=[
                                    'BRA',
                                    'MEX',
                                    'RCAM',
                                    'RSAM',
                              ],
                              weight=weight
                              )

    df_we = df.aggregate_region(variables,
                              region='Western Europe and Other States (UN-R5)',
                              subregions=[
                                        'CAN',
                                        'OCE',
                                        'TUR',
                                        'USA',
                                        'WEU',
                              ],
                              weight=weight
                              )

    
    df_un_groups = pyam.concat([df_africa, df_asia, df_ee, df_latin, df_we])
    return df_un_groups



In [6]:
df_mid_clean = pyam.IamDataFrame(df_unclean)
percentage_vars = df_mid_clean.filter(unit='% of GDP').variable
un_percentage = compute_UN_groups(df_mid_clean, percentage_vars, weight='GDP|Climate Damage Corrected')
absolute_vars = df_mid_clean.filter(unit='trillion USD_2010/yr').variable
un_absolute = compute_UN_groups(df_mid_clean, absolute_vars)
df_world = df_mid_clean.filter(region='World')
df_clean = pyam.concat([un_percentage, un_absolute, df_world])
df_clean.timeseries()

2020  \
model  scenario region         variable                                   unit                               
MIMOSA LIFE-TP  Africa (UN-R5) Avoided Damages                            trillion USD_2010/yr    0.000000   
                               Avoided Damages|Relative                   % of GDP                0.000000   
                               Damage Costs                               trillion USD_2010/yr    0.000000   
                               GDP|Climate Damage Corrected               trillion USD_2010/yr    5.568112   
                               GDP|Climate Damage Corrected|Loss          trillion USD_2010/yr    0.000000   
...                                                                                                    ...   
       TECH-TP  World          Damage Costs|Indirect                      % of GDP                0.000000   
                               Damage Costs|Relative                      % of GDP                0.000000   
                               GDP|Climate Damage Corrected               trillion USD_2010/yr  102.473673   
                               GDP|Climate Damage Corrected|Loss          trillion USD_2010/yr    0.000000   
                               GDP|Climate Damage Corrected|Loss|Relative % of GDP                0.000000   

                                                                                                      2030  \
model  scenario region         variable                                   unit                               
MIMOSA LIFE-TP  Africa (UN-R5) Avoided Damages                            trillion USD_2010/yr    0.026267   
                               Avoided Damages|Relative                   % of GDP                0.289622   
                               Damage Costs                               trillion USD_2010/yr    0.097668   
                               GDP|Climate Damage Corrected               trillion USD_2010/yr    8.944009   
                               GDP|Climate Damage Corrected|Loss          trillion USD_2010/yr    0.122387   
...                                                                                                    ...   
       TECH-TP  World          Damage Costs|Indirect                      % of GDP                0.134545   
                               Damage Costs|Relative                      % of GDP                0.576325   
                               GDP|Climate Damage Corrected               trillion USD_2010/yr  149.935931   
                               GDP|Climate Damage Corrected|Loss          trillion USD_2010/yr    1.073481   
                               GDP|Climate Damage Corrected|Loss|Relative % of GDP                0.710870   

                                                                                                      2040  \
model  scenario region         variable                                   unit                               
MIMOSA LIFE-TP  Africa (UN-R5) Avoided Damages                            trillion USD_2010/yr    0.246750   
                               Avoided Damages|Relative                   % of GDP                1.582574   
                               Damage Costs                               trillion USD_2010/yr    0.297291   
                               GDP|Climate Damage Corrected               trillion USD_2010/yr   15.200790   
                               GDP|Climate Damage Corrected|Loss          trillion USD_2010/yr    0.386010   
...                                                                                                    ...   
       TECH-TP  World          Damage Costs|Indirect                      % of GDP                0.294655   
                               Damage Costs|Relative                      % of GDP                1.099253   
                               GDP|Climate Damage Corrected               trillion USD_2010/yr  194.932655   
                              

In [7]:
df_clean.to_excel('../../../data/TOD/model_results/to_share/MIMOSA_to_share.xlsx')

In [8]:
def plot_data_individual_figures(df, title_dict=None, path_dict=None, variables=None,):
    
    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
        "REF-v2": "black",
        "LIFE-TP-v2": "orange",
        "TECH-TP-v2": "magenta",
    }

    regions_dict = {
        'Africa (UN-R5)': 'Africa',
        'Asia and the Pacific (UN-R5)': 'AatP',
        'Eastern Europe (UN-R5)': 'EE',
        'Latin America and Caribbean (UN-R5)': 'LaaC',
        'Western Europe and Other States (UN-R5)': 'WeoS'
                }
    
    if variables is None:
            variables = df.variable

    for region in list(regions_dict.keys()):
        df_plotting = df.filter(year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050], region=region).timeseries().reset_index()

        
        df_selected = df_plotting[df_plotting['variable'].isin(variables)]

        df_compare_data = pd.melt(df_selected, id_vars=['model',
                                    'scenario',
                                    'region',
                                    'variable',
                                    'unit'
                                        ], var_name='year', value_name='value')
        df_compare_data['year'] = df_compare_data['year'].astype(int)

        

        for variable in variables: # ,variables
            # Use Seaborn to create the plot with markers and lines
            data = df_compare_data[df_compare_data["variable"] == variable]
            fig, ax = plt.subplots(figsize=(10, 6))  # Set figure size
            sns.lineplot(
                data=data,
                x="year",
                y="value",
                hue="scenario",
                # style="model",
                dashes=True,
                palette=scenario_colors,
            )

            # Set labels and title
            plt.xlabel("Year")
            plt.ylabel(data["unit"].iloc[0])  # Set the y-axis label to the unit
            # Set y-axis lower limit to 0
            ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
            # Mid point of left and right x-positions
            mid = (fig.subplotpars.right + fig.subplotpars.left)/2
            if title_dict:
                plt.suptitle(title_dict[variable], x=0.4)  # Set the title to the variable
                plt.title(region)
            else:
                fig.suptitle(variable, fontsize=16)
            # Show legend and grid
            plt.legend(title="Legend", bbox_to_anchor=(1.3, 1))
            plt.grid(True)
            # Adjust layout to ensure legend fits within the saved image
            plt.tight_layout()
            # root_variable = variable.split("|")[0].replace(" ", "_")
            if path_dict:
                variable_name = variable.replace(" > 25 μg/m3", "").replace("|", "_").replace(" ", "_").replace("/", "")
                plt.savefig(
                    f"../../../plots/TOD/chpt_20/Regional/{regions_dict[region]}/{path_dict[variable]}/{variable_name}.png"
                )
                plt.close()

In [9]:
path_dict = {
    'Avoided Damages':'Macro_Economy',
    'Avoided Damages|Relative':'Macro_Economy',
    'Damage Costs':'Macro_Economy',
    # 'Damage Costs|Indirect':'Macro_Economy',
    # 'Damage Costs|Relative':'Macro_Economy',
    'GDP|Climate Damage Corrected':'Macro_Economy',
    'GDP|Climate Damage Corrected|Loss':'Macro_Economy',
    'GDP|Climate Damage Corrected|Loss|Relative':'Macro_Economy'}


# plot_data_individual_figures(df_clean, path_dict=path_dict, variables=path_dict.keys())

# plot_data_3_x_3_figures(df_clean)

In [10]:
def plot_data_3_x_3_figures(df, title_dict=None, path_dict=None, variables=None,):

    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
        "REF-v2": "black",
        "LIFE-TP-v2": "orange",
        "TECH-TP-v2": "magenta",
    }

    df_plotting = df.filter(year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050]).timeseries().reset_index()
    df_compare_data = pd.melt(df_plotting, id_vars=['model',
                                'scenario',
                                'region',
                                'variable',
                                'unit'
                                    ], var_name='year', value_name='value')
    df_compare_data['year'] = df_compare_data['year'].astype(int)

    if variables is None:
        variables = df.variable

    for variable in variables:
        df_var = df_compare_data[df_compare_data["variable"] == variable]
        # Get unique regions for creating subplots
        unique_regions = df_var['region'].unique()

        # Create subplots based on the number of unique regions
        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))

        # Initialize lists to store handles and labels for the legend
        legend_handles = []
        legend_labels = []

        # Iterate through each region and plot in respective subplot
        for idx, region in enumerate(unique_regions):
            row_idx = idx // 3
            col_idx = idx % 3
            
            ax = axes[row_idx, col_idx]  # Select the current subplot
            
            region_data = df_var[df_var['region'] == region]  # Filter data for the current region
            
            # Plot lineplots for each region
            lineplot = sns.lineplot(data=region_data, 
                                    x="year", 
                                    y="value", 
                                    hue="scenario", 
                                    # style='model', 
                                    palette=scenario_colors, 
                                    linewidth=2.5, 
                                    ax=ax)
            
            # Customize ticks and labels for the current subplot
            ax.set_ylabel(region_data["unit"].iloc[0], fontsize=14)  # Set the y-axis label to the unit
            # Set y-axis lower limit to 0
            ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
            ax.set_xlabel('', fontsize=1)
            ax.set_title(f"{region}", fontsize=14)  # Set the title to the region
            ax.set_xticks([2010, 2020, 2030, 2040, 2050])
            ax.set_xticklabels([2010, 2020, 2030, 2040, 2050], 
                            rotation=45, fontsize=14)
            
            # Increase fontsize of y-axis tick labels
            ax.tick_params(axis='y', labelsize=14)

            # Add the lineplot to the legend manually
            legend_handles.append(lineplot)
            legend_labels.append(region)
            # Disable the legend for individual subplots
            ax.legend().set_visible(False)
            ax.grid(True)

            # Add a gray background for the 'World' region subplot
            if region == 'World':
                ax.set_facecolor(color='#ededed')

        # Show legend and grid for the current subplot
        if title_dict:
            fig.suptitle(f"{title_dict[variable]}", fontsize=16)
        else:
            fig.suptitle(variable, fontsize=16)

        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels, title="Legend", bbox_to_anchor=(1.12, 0.55))
        plt.tight_layout()  # Adjust subplots to prevent overlap
        
        if path_dict:
            root_variable = variable.split("|")[0].replace(" ", "_")
            variable_name = variable.replace(" > 25 μg/m3", "").replace("|", "_").replace(" ", "_").replace("/", "")
            # plt.savefig(f"../../../plots/SOD/chpt_20/Regional/{path_dict[variable]}/{variable_name}.png", bbox_inches='tight')
            # plt.close()